# render-metro-views

Display the views that the metro-edge application is providing. 

#### Before you begin,  verify that you've executed [build-metro-application](./build-metro-application.jupyter-py36.ipynb) notebook.
The build-metro-application notebook composes and submits the metro application that recieves status from the edge, this notebook renders data from the metro application.

The metro application receives two types of messages on a topic from the edge. The 'ClassificationMetrics' messages provide statistics on the scoring on the edge, these messages are aggregated for time averaging. The 'UncertainImages' messages contains images that have a lower-than-acceptable confidence rating that require deeper analysis and possible manual labelling.

This notebook renders the data processed by the metro application. 

In [ ]:
%matplotlib inline
%gui asyncio
import urllib3
import time
import threading
import base64
import sys
import IPython
#from IPython import display  ## DO NOT use interferes with display()
import ipywidgets as widgets
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
from IPython.core.debugger import set_trace
## 
from icpd_core import icpd_util
from streamsx.rest_primitives import Instance
from streamsx.topology import context

if '/project_data/data_asset' not in sys.path:
    sys.path.insert(0, '/project_data/data_asset')
import metrorender


In [ ]:
# Cell to grab Streams instance config object and REST reference
urllib3.disable_warnings()
STREAMS_INSTANCE_NAME = "edge"
streams_cfg=icpd_util.get_service_instance_details(name=STREAMS_INSTANCE_NAME)
streams_cfg[context.ConfigParams.SSL_VERIFY] = False
instance = Instance.of_service(streams_cfg)

# Verify that the Metro-Edge Streams Application is active and healthy

In [ ]:
# Verify that the job is healthy/up before proceding..
#
urllib3.disable_warnings()
# list the active jobs
print("Active Jobs:")
for job in instance.get_jobs():
    print("  ", job.name, job.health)


# Bring up the live Queues of Views
- ClassificationMetrics
- WindowUncertain

In [ ]:
# WindowUncertain - queue
output_WindowUncertain = widgets.Output()
display(output_WindowUncertain )
WindowUncertain_vtq = metrorender.view_to_queue(instance, "WindowUncertain", output_WindowUncertain)
WindowUncertain_vtq.start()  # start
#print("windowUncertain_thread\n\t alive:{}\n\t event:{}\n\t queue depth:{}".format(WindowUncertain_vtq.thread.is_alive(), WindowUncertain_vtq.event.is_set(), len(WindowUncertain_vtq.tuples)))
# WindowUncertain_vtq.event.clear()  # emergency kill

# UncertainPredictions - queue
output_UncertainPredictions = widgets.Output()
display(output_UncertainPredictions )
UncertainPredictions_vtq = metrorender.view_to_queue(instance, "UncertainPredictions", output_UncertainPredictions)
UncertainPredictions_vtq.start()  # start
#print("UncertainPredictions_thread\n\t alive:{}\n\t event:{}\n\t queue depth:{}".format(UncertainPredictions_vtq.thread.is_alive(), UncertainPredictions_vtq.event.is_set(), len(UncertainPredictions_vtq.tuples)))
# UncertainPredictions_vtq.event.clear()  # emergency kill

# Specify the cameras

Discover the cameras that are available by waiting for events for 10 seconds, during which time at least one ClassificationMetrics message should have arrived, with the list of
currently active cameras, which is displayed.  If you wish to override the discovered set of cameras, to only show metrics from a subset, set the ACTIVE_CAMERAS to that subset.


In [ ]:
import json
# Wait a bit for the camera metrics to come in.
time.sleep(10)
chunks = WindowUncertain_vtq.tuples.copy()
ACTIVE_CAMERAS = set({})
for chunk in chunks:
    for tups in chunk:
        #tup = json.loads(tups)
        tup = tups
        key_list = tup['camera_metrics'].keys()
        ACTIVE_CAMERAS.add(list(key_list)[0])

# Uncomment to override the detected cameras
#ACTIVE_CAMERAS = {'Camera-X', 'Camera-Y'}

ACTIVE_CAMERAS

# Per-Camera Digit Prediction Metrics

For each camera, the current image throughput is shown, along with a graph showing the distribution of all images in the recent interval that were predicted to be each digit.  The two bars for each digit show the certain vs. uncertain predictions for each digit.

The graphs will continue to update based on the most recent metrics until you Interrupt the kernel, to move on to the next cell.

In [ ]:
#%%script false --no-raise-error
idx = 1
while (len(WindowUncertain_vtq.tuples) < 5):
    print("priming{}".format(idx*"."),end="\n")
    idx += 1
    time.sleep(2)
print("primed           ")
output_graphs = widgets.Output()
display(output_graphs)
synchronous_event = threading.Event()
synchronous = metrorender.deque_synchronous(WindowUncertain_vtq.tuples, count=5, debug=False)
rwu =  metrorender.RenderWindowUncertain(output_graphs, ACTIVE_CAMERAS)
try: 
    rwu.render(synchronous,synchronous_event)
except KeyboardInterrupt:
    print("Interrupt caught...")
    rwu.class_status_widget.value = "Interupt * Finished"
rwu.class_status_widget.value = "Rendering - Finished"
 

# Display sampled set uncertain images

As images are scored, images where the model's prediction confidence for any given digit is too low are returned
to the Metro-edge. There, these images could be manually scored, and potentially used to build a more robust model.

Below is a sampling of the uncertain images that were returned to the metro-edge recently.  They will continue updating
until the kernel is Interrupted, to move on to the next cell.


In [ ]:
#%%script false --no-raise-error
# Un-threaded version
output_uncertain = widgets.Output()
display(output_uncertain)
rui = metrorender.RenderUncertainImages(output_uncertain)
rui.stop_button.description = "Use Interrupt"
rui.stop_button.tooltip = "Use Interrupt Kernel above"
active = True
try:
    while active:
        try:
            rui.display_view(UncertainPredictions_vtq.tuples.pop(), "live")
            time.sleep(.7) # slow down - prevent widget overrun
        except IndexError:
            time.sleep(3)
except KeyboardInterrupt:
    active = False
    rui.interrupt_stopped("Review displayed Images")

# Correction Station

The current model is not perfect.  When it encounters images that it cannot classify with confidence, these images are sent down the 'UncertainPrediction' view. In order to improve the model, the questionable images need to be assigned a value and added into the training data for the next round of model regeneration. The purpose of this
dashboard is to review the questionable images and either accept the predicted label, or adjust the label as necessary.

In an environment where the images are the output of a camera on the edge, say in a manufacturing line, not all incorrect predictions are the result of a poor model: in some cases the camera may be faulting, or misaligned, or the lighting may have been lost, etc.  For some of these cases, the model may still be improved to be more robust in these error situations, but in other cases, the root problem should be fixed, but the incorrect images shouldn't be used to re-train the model, and so should be discarded.

In a full solution, mocked up here, the questionable images are displayed to the left, and their per-digit scores (according to the current model) are displayed to the right, with a default predicted label chosen.  The user could adjust the label if they are confident in the correct one, or ask for a second opinion, or declare that there is a camera issue, or some other problem.  As each image is handled, the next arrow at the bottom can be used to move on to the next image to manually label.  When a particular manual labeling session is complete, the "Training Upload" button might be used to send the manually labeled images to some database that will be used when the model is next re-built.


In [ ]:
#%%script false --no-raise-error

while len(UncertainPredictions_vtq.tuples)< 20:
    time.sleep(3)
    print(" - waiting for events ...")
snapShot = UncertainPredictions_vtq.tuples.copy()
cd = metrorender.CorrectionDashboard()
cd.render_review(snapShot)